In [16]:
from plotly import express as px

In [17]:
import plotly.graph_objects as go
import polars as pl
import torch

from cci.utils import project_dir

# res_dir = project_dir() / "results/CNN/32"
# res_dir = project_dir() / "results/CNN/40"
# res_dir = project_dir() / "results/CNN/47"
# res_dir = project_dir() / "results/CNN/65"
# res_dir = project_dir() / "results/CNN/101"
# res_dir = project_dir() / "results/CNN/114"
# Her++++++++++++++++++++++++++++++++++++++++++++++++4444
# res_dir = project_dir() / "results/CNN/115"4
# res_dir = project_dir() / "results/CNN/136"
# res_dir = project_dir() / "results/CNN/153"
# res_dir = project_dir() / "results/CNN/161"
# res_dir = project_dir() / "results/CNN/169"
# res_dir = project_dir() / "results/CNN/171"
# res_dir = project_dir() / "results/CNN/174"

res_dir = project_dir() / "results/CNN_vft/91"
res_dir = project_dir() / "results/CNN_full/66"
res_dir = project_dir() / "results/CNN_bigger_kernel_full_fixed/160"
res_dir = project_dir() / "results/CNN_bigger_kernel_vft_fixed/196"
res_dir = project_dir() / "results/CNN_bigger_kernel_as_fixed/151"
res_dir = project_dir() / "results/CNN_bigger_kernel_full_fixed/349"
# model = torch.load(res_dir / "0_model.pt", map_location=torch.device("cpu"))
# print()

In [18]:
test_df = pl.read_csv(res_dir / "*_test.csv").drop("epoch")
test_df

acc,loss,f1,auroc,precision,recall,specificity,bac,TP,FP,FN,TN
f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64
0.700935,0.518102,0.813953,0.652299,0.823529,0.804598,0.25,0.527299,5,15,17,70


In [19]:
import numpy as np

from cci.metrics import confusion_matrix

# BUG: Fixed now but..
test_len = 107  # length of test set
for tp, fp, fn, tn in test_df.select(["TP", "FP", "FN", "TN"]).rows():
    tp = test_len - fp - fn - tn
    cm = np.array([[tp, fp], [fn, tn]]).astype("float")
    normalized_cm = cm / cm.sum(axis=1)[:, np.newaxis]
    fig = px.imshow(
        normalized_cm,
        labels=dict(x="Predicted", y="Actual"),
        x=["Good (0)", "Bad (1)"],
        y=["Good (0)", "Bad (1)"],
        text_auto=True,
    )
    fig.show()

In [20]:
def get_df(group) -> pl.DataFrame:
    df = pl.read_csv(res_dir / f"0_{group}.csv").with_columns(
        pl.lit(0).alias("fold"),
        pl.lit(group).alias("group"),
        pl.lit(f"{group}_0").alias("identifier"),
    )

    try:
        for i in range(1, 5):
            df = df.vstack(
                pl.read_csv(res_dir / f"{i}_{group}.csv").with_columns(
                    pl.lit(i).alias("fold"),
                    pl.lit(group).alias("group"),
                    pl.lit(f"{group}_{i}").alias("identifier"),
                )
            )
    except:
        pass

    return df


df = get_df("train")

df = df.vstack(get_df("val"))
df

epoch,acc,loss,f1,auroc,precision,recall,specificity,bac,TP,FP,FN,TN,fold,group,identifier
i64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64,i32,str,str
1,0.666235,0.245514,0.78956,0.476779,0.823129,0.758621,0.22963,0.494125,31,104,154,484,0,"""train""","""train_0"""
2,0.614489,0.241952,0.736283,0.53576,0.845528,0.652038,0.437037,0.544537,59,76,222,416,0,"""train""","""train_0"""
3,0.663648,0.241593,0.779287,0.551144,0.85,0.719436,0.4,0.559718,54,81,179,459,0,"""train""","""train_0"""
4,0.683053,0.241274,0.799673,0.561262,0.835897,0.766458,0.288889,0.527673,39,96,149,489,0,"""train""","""train_0"""
5,0.491591,0.240817,0.59857,0.586892,0.859238,0.459248,0.644444,0.551846,87,48,345,293,0,"""train""","""train_0"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
728,0.715789,0.575033,0.798507,0.7633,0.89916,0.718121,0.707317,0.712719,29,12,42,107,0,"""val""","""val_0"""
729,0.710526,0.564631,0.802867,0.740219,0.861538,0.751678,0.560976,0.656327,23,18,37,112,0,"""val""","""val_0"""
730,0.726316,0.556347,0.811594,0.735636,0.88189,0.751678,0.634146,0.692912,26,15,37,112,0,"""val""","""val_0"""


In [21]:
# fig = px.line(df, x="epoch", y="loss", line_group="identifier", color="group", height=800)


def avg_plot(metric: str):
    fig = go.Figure()
    for x, color in (["train", "blue"], ["val", "red"]):
        df = get_df(x)
        loss_df = (
            df.group_by("epoch", maintain_order=True)
            .agg(pl.col(metric).mean().alias("avg"), pl.col(metric).std().alias("std"))
            .to_pandas()
        )
        fig.add_trace(
            go.Scatter(
                name=f"Avg {x} {metric}",
                x=loss_df["epoch"],
                y=loss_df["avg"],
                mode="lines",
            )
        )
        fig.add_trace(
            go.Scatter(
                name=f"{x}upper bound",
                x=loss_df["epoch"],
                y=loss_df["avg"] + loss_df["std"],
                line=dict(width=0),
                marker=dict(color=color),
                showlegend=False,
            )
        )
        fig.add_trace(
            go.Scatter(
                name=f"{x}lower bound",
                x=loss_df["epoch"],
                y=loss_df["avg"] - loss_df["std"],
                line=dict(width=0),
                marker=dict(color=color),
                fill="tonexty",
                showlegend=False,
            ),
        )
    fig.update_layout(hovermode="x", height=800)
    fig.show()


avg_plot("loss")

In [22]:
avg_plot("bac")

In [23]:
avg_plot("acc")

In [24]:
avg_plot("f1")

In [25]:
avg_plot("auroc")